# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [40]:
#!pip install imread

In [41]:
import numpy as np
import os
#from scipy.misc import imread, imresize
import datetime
from skimage import io
from random import shuffle

We set the random seed so that the results don't vary drastically.

In [42]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
from keras.layers import Dense, Dropout, Flatten
import tensorflow as tf
tf.set_random_seed(30)
import keras

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [43]:
source_path = "./data/Project_data"

In [44]:
train_doc = np.random.permutation(open(source_path +'/train.csv').readlines())
val_doc = np.random.permutation(open(source_path +'/val.csv').readlines())
batch_size = 10

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [45]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = np.random.randint(0,30,size=10)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = 2
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,15,100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = io.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                     
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    if image.shape[0] > 100 and image.shape[1] > 100:
                        h, w, _ = image.shape
                        image = image[int(h/2)-50:int(h/2)+50, int(w/2)-50:int(w/2)+50, : ]

                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255  #R
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255  #G
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255  #B
                    
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [46]:
curr_dt_time = datetime.datetime.now()
train_path = source_path+'/train'
val_path = source_path+ '/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [47]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers


img_rows, img_cols, img_depth = 15, 100, 100
input_shape = (img_rows, img_cols, img_depth, 3)

conv3Dmodel = Sequential()
conv3Dmodel.add(Conv3D(32, kernel_size=(2, 2, 2),
                 activation='relu',
                 input_shape=input_shape)) # input shape = (img_rows, img_cols, 1)

# second conv layer
conv3Dmodel.add(Conv3D(64, kernel_size=(2, 2, 2), 
                 activation='relu'))
conv3Dmodel.add(MaxPooling3D(pool_size=(2, 2, 2)))
conv3Dmodel.add(Dropout(0.25))

# flatten and put a fully connected layer
conv3Dmodel.add(Flatten())
conv3Dmodel.add(Dense(128, activation='relu')) # fully connected
conv3Dmodel.add(Dropout(0.5))

# softmax layer
conv3Dmodel.add(Dense(5, activation='softmax'))

# model summary
conv3Dmodel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_7 (Conv3D)            (None, 14, 99, 99, 32)    800       
_________________________________________________________________
conv3d_8 (Conv3D)            (None, 13, 98, 98, 64)    16448     
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 6, 49, 49, 64)     0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 6, 49, 49, 64)     0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 921984)            0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               118014080 
_________________________________________________________________
dropout_8 (Dropout)          (None, 128)               0         
__________

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [48]:
optimiser = keras.optimizers.Adadelta()
conv3Dmodel.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (conv3Dmodel.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_7 (Conv3D)            (None, 14, 99, 99, 32)    800       
_________________________________________________________________
conv3d_8 (Conv3D)            (None, 13, 98, 98, 64)    16448     
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 6, 49, 49, 64)     0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 6, 49, 49, 64)     0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 921984)            0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               118014080 
_________________________________________________________________
dropout_8 (Dropout)          (None, 128)               0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [49]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [50]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=5, min_lr=0.001)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [51]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [52]:
conv3Dmodel.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=2,use_multiprocessing=True, initial_epoch=0)

Epoch 1/10


/home/sanmati/anaconda3/envs/AB_GPU/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Source path =  ./data/Project_data/val ; batch size = 10
Source path =  ./data/Project_data/val ; batch size = 10
Source path =  ./data/Project_data/train ; batch size = 10
Source path =  ./data/Project_data/train ; batch size = 10
67/67 [==============================] - 16s 242ms/step - loss: 4.7769 - categorical_accuracy: 0.3701 - val_loss: 1.5632 - val_categorical_accuracy: 0.3200

Epoch 00001: saving model to model_init_2018-10-1618_52_43.555081/model-00001-4.77692-0.37015-1.56320-0.32000.h5
Epoch 2/10
67/67 [==============================] - 11s 171ms/step - loss: 1.2401 - categorical_accuracy: 0.5149 - val_loss: 2.6226 - val_categorical_accuracy: 0.2400

Epoch 00002: saving model to model_init_2018-10-1618_52_43.555081/model-00002-1.24009-0.51493-2.62265-0.24000.h5
Epoch 3/10
67/67 [==============================] - 13s 198ms/step - loss: 0.8902 - categorical_accuracy: 0.7075 - val_loss: 1.2521 - val_categorical_accuracy: 0.5200

Epoch 00003: saving model to model_init_2018-10-1